# Flan-T5 Model: Distillation and Pruning

This notebook demonstrates the process of distilling and pruning a Flan-T5 model using real datasets.

In [1]:
!pip install transformers datasets tensorflow flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

## Import Libraries and Load Dataset

In [2]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
import numpy as np
import os

# Load the dataset
dataset = load_dataset('wmt16', 'de-en')
train_dataset = dataset['train'].select(range(1000))

# Load teacher and student models
teacher_model = TFAutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
student_model = TFAutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-small')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Tokenization

In [3]:

# Tokenize the dataset
def preprocess_function(examples):
    inputs = [f"Translate from English to German: {ex['en']}" for ex in examples['translation']]
    targets = [ex['de'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
train_dataset.set_format('tensorflow', columns=['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

# Convert datasets to tf.data.Dataset
def dataset_to_tfdata(dataset):
    def generator():
        for example in dataset:
            yield {key: example[key].numpy() for key in example.keys()}

    return tf.data.Dataset.from_generator(generator, output_signature={
        'input_ids': tf.TensorSpec(shape=(128,), dtype=tf.int32),
        'attention_mask': tf.TensorSpec(shape=(128,), dtype=tf.int32),
        'labels': tf.TensorSpec(shape=(128,), dtype=tf.int32),
        'decoder_input_ids': tf.TensorSpec(shape=(128,), dtype=tf.int32)
    })

train_tf_dataset = dataset_to_tfdata(train_dataset).batch(8)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Calculating Teacher logits

In [4]:

# Calculate teacher logits and save in chunks
logits = []
chunk_size = 50  # Adjust chunk size according to your memory constraints
chunk_counter = 0
os.makedirs('teacher_logits_chunks', exist_ok=True)

for i, batch in enumerate(train_tf_dataset):
    logit = teacher_model(batch, training=False).logits
    logits.append(logit)

    if len(logits) * logit.shape[0] >= chunk_size:
        np.save(f'teacher_logits_chunks/teacher_logits_chunk_{chunk_counter}.npy', tf.concat(logits, axis=0).numpy())
        logits = []
        chunk_counter += 1

# Save remaining logits
if logits:
    np.save(f'teacher_logits_chunks/teacher_logits_chunk_{chunk_counter}.npy', tf.concat(logits, axis=0).numpy())

# Load saved logits for training
def load_teacher_logits():
    logits = []
    for chunk_file in sorted(os.listdir('teacher_logits_chunks')):
        chunk_logits = np.load(os.path.join('teacher_logits_chunks', chunk_file))
        logits.append(chunk_logits)
    return tf.convert_to_tensor(np.concatenate(logits, axis=0))

teacher_logits = load_teacher_logits()


## Creating final dataset

In [5]:

# Combine teacher logits with the training data
def add_teacher_logits_to_dataset(dataset, teacher_logits):
    new_dataset = []
    for i, example in enumerate(dataset):
        input_ids = example['input_ids']
        decoder_input_ids = example['decoder_input_ids']
        logit = teacher_logits[i]
        new_dataset.append((input_ids, decoder_input_ids, logit))
    return new_dataset

train_dataset_final = add_teacher_logits_to_dataset(train_dataset, teacher_logits)

# Convert to tf.data.Dataset
def create_tf_dataset(dataset):
    inputs = {'input_ids': [], 'decoder_input_ids': []}
    logits = []
    for input_ids, decoder_input_ids, logit in dataset:
        inputs['input_ids'].append(input_ids)
        inputs['decoder_input_ids'].append(decoder_input_ids)
        logits.append(logit)

    inputs = {key: tf.convert_to_tensor(value) for key, value in inputs.items()}
    logits = tf.convert_to_tensor(logits)
    return tf.data.Dataset.from_tensor_slices((inputs, logits))

train_tf_dataset = create_tf_dataset(train_dataset_final).batch(4)


## Training the student model

In [6]:

# Define custom loss function
def custom_loss(y_true, y_pred):
    return tf.keras.losses.KLDivergence()(y_true, y_pred)

# Compile the model
student_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss=custom_loss)

# Train the student model
student_model.fit(train_tf_dataset, epochs=3)

# Save the distilled model
student_model.save_pretrained('./distilled_model')


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
250/250 [==============================] - 136s 145ms/step - loss: -0.0516
Epoch 2/3
250/250 [==============================] - 23s 92ms/step - loss: -0.0518
Epoch 3/3
250/250 [==============================] - 23s 92ms/step - loss: -0.0518


In [7]:
import gc

tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()
del train_tf_dataset
del train_dataset_final
del teacher_logits
del teacher_model
del student_model
del logits
del dataset
gc.collect()


76

## Prune the Model

In [8]:
import tf_keras
import numpy as np
import os

def prune_layer(layer, pruning_factor=0.2):
    if isinstance(layer, tf_keras.src.layers.core.dense.Dense):
        weights = layer.get_weights()[0]
        abs_weights = np.abs(weights)
        threshold = np.percentile(abs_weights, pruning_factor * 100)
        mask = abs_weights >= threshold
        mask = tf.cast(mask, weights.dtype)
        new_weights = weights * mask
        layer.set_weights([new_weights])
        return layer

def prune_model(model, pruning_factor=0.2):
    for layer in model.encoder._flatten_layers():
        layer =  prune_layer(layer, pruning_factor)
    for layer in model.decoder._flatten_layers():
        layer =  prune_layer(layer, pruning_factor)
    layer = prune_layer(model.layers[-1], pruning_factor/2)
    return model

def get_zero_and_nonzero_params(model):
  total_zero_params = 0
  total_nonzero_params = 0
  for weight in model.get_weights():
    rows = weight.shape[0]
    cols = weight.shape[1] if len(weight.shape) > 1 else 1
    total_params = rows * cols
    total_params_non_zero = tf.math.count_nonzero(weight).numpy()
    total_zero_params += total_params - total_params_non_zero
    total_nonzero_params += total_params_non_zero
  return total_zero_params, total_nonzero_params

## Import Libraries and Load Dataset

In [9]:
from transformers import TFAutoModelForSeq2SeqLM
import tensorflow as tf
model = TFAutoModelForSeq2SeqLM.from_pretrained('./distilled_model')
pruned_model = prune_model(model, pruning_factor=0.1)
zero_params_pruned, total_nonzero_params_pruned = get_zero_and_nonzero_params(pruned_model)
print(f"Total number of zero parameters after pruning: {zero_params_pruned}, and nonzero parameters after pruning: {total_nonzero_params_pruned}")
pruned_model.save_pretrained('./pruned_model')
del model
del pruned_model

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at ./distilled_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Total number of zero parameters after pruning: 5226525, and nonzero parameters after pruning: 71734627


In [10]:
import gc
gc.collect()

189667

## Write the Chatbot


In [11]:
%%writefile flask_chatbot_app.py

import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from flask import Flask, request, jsonify
import numpy as np
import os

# Flask app
app = Flask(__name__)
model_path = "./pruned_model"
# Load the model and tokenizer using Hugging Face
model_name = "google/flan-t5-base"

# Explicitly load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True)
conversation_history = []

# Create the pipeline
chatbot = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
sentiment_analysis = pipeline("sentiment-analysis")

def adjust_prompt_based_on_sentiment(prompt):
    sentiment = sentiment_analysis(prompt)[0]
    if sentiment['label'] == 'NEGATIVE':
        prompt = f"User seems upset. Respond in a calming manner: {prompt}"
    return prompt

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')

    # Add user input to conversation history
    conversation_history.append({"role": "User", "message": user_input})

    # Adjust prompt based on sentiment
    adjusted_input = adjust_prompt_based_on_sentiment(user_input)

    # Prepare the full conversation as the prompt
    full_conversation = ""
    for turn in conversation_history:
        full_conversation += f"{turn['role']}: {turn['message']} "

    # Generate response using the fine-tuned model
    inputs = tokenizer(full_conversation, return_tensors="tf", max_length=1024, truncation=True, padding=True)
    outputs = model.generate(**inputs, max_new_tokens=250)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Add assistant response to conversation history
    conversation_history.append({"role": "Assistant", "message": response_text})

    return jsonify({"response": response_text})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Writing flask_chatbot_app.py
